In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle
import networkx as nx
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint


# Define the file path in Google Drive
file_path = '/content/drive/MyDrive/Yannis/all_variables.pkl'

def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

# Load Data and Initialize Variables
year_to_load = 2016
loaded_variables = load_variables(year_to_load)

Eora_pp_FD = loaded_variables['Eora_pp_FD']
labels_FD = loaded_variables['labels_FD']
Eora_pp_Q = loaded_variables['Eora_pp_Q']
labels_Q = loaded_variables['labels_Q']
Eora_pp_QY = loaded_variables['Eora_pp_QY']
Eora_pp_T = loaded_variables['Eora_pp_T']
labels_T = loaded_variables['labels_T']
Eora_pp_VA = loaded_variables['Eora_pp_VA']
labels_VA = loaded_variables['labels_VA']
Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
xout = loaded_variables['xout']
A = loaded_variables['A']
f = loaded_variables['f']
I = loaded_variables['I']
L = loaded_variables['L']
paths = loaded_variables['paths']
sector_labels = loaded_variables['sector_labels']
total_impact = loaded_variables['total_impact']
centrality = loaded_variables['centrality']
bonacich_centrality = loaded_variables['bonacich_centrality']

Mounted at /content/drive


In [ ]:
# Initial conditions for production and emissions
initial_production = Eora_pp_T_matrix  # Initial production values
initial_emissions = Eora_pp_Q_emissions_matrix  # Initial emissions values

# Combine into a single initial conditions vector
initial_conditions = np.concatenate([initial_production, initial_emissions])

def feedback_production(production, demand_matrix):
    return np.dot(demand_matrix, production)

def feedback_emissions(production, emissions_factors):
    return emissions_factors * production

def system_dynamics(y, t, demand_matrix, emissions_factors, external_shocks):
    num_sectors = len(demand_matrix)
    production = y[:num_sectors]
    emissions = y[num_sectors:]

    production_change = feedback_production(production, demand_matrix) + external_shocks(t)
    emissions_change = feedback_emissions(production, emissions_factors)

    return np.concatenate([production_change, emissions_change])

def external_shocks(t):
    # Example external shock function
    return np.sin(t) * np.ones(10)

# Time vector for simulation
t = np.linspace(0, 10, 100)

# Demand matrix and emissions factors (example values)
# Check the type of Eora_pp_FD and convert to numpy matrix if necessary
if not isinstance(Eora_pp_FD, np.ndarray):
    Eora_pp_FD = np.array(Eora_pp_FD)
demand_matrix = Eora_pp_FD
emissions_factors = f

# Perform the simulation
result = odeint(system_dynamics, initial_conditions, t, args=(demand_matrix, emissions_factors, external_shocks))

# Extract results
production_result = result[:, :10]
emissions_result = result[:, 10:]

# Plot results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(t, production_result)
plt.title('Production over time')
plt.xlabel('Time')
plt.ylabel('Production')

plt.subplot(1, 2, 2)
plt.plot(t, emissions_result)
plt.title('Emissions over time')
plt.xlabel('Time')
plt.ylabel('Emissions')

plt.tight_layout()
plt.show()


ValueError: Initial condition y0 must be one-dimensional.

One Chunk attempt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle
import networkx as nx
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Define the file path in Google Drive
file_path = '/content/drive/MyDrive/Yannis/all_variables.pkl'

def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

# Load Data and Initialize Variables
year_to_load = 2016
loaded_variables = load_variables(year_to_load)

Eora_pp_FD = loaded_variables['Eora_pp_FD']
labels_FD = loaded_variables['labels_FD']
Eora_pp_Q = loaded_variables['Eora_pp_Q']
labels_Q = loaded_variables['labels_Q']
Eora_pp_QY = loaded_variables['Eora_pp_QY']
Eora_pp_T = loaded_variables['Eora_pp_T']
labels_T = loaded_variables['labels_T']
Eora_pp_VA = loaded_variables['Eora_pp_VA']
labels_VA = loaded_variables['labels_VA']
Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
xout = loaded_variables['xout']
A = loaded_variables['A']
f = loaded_variables['f']
I = loaded_variables['I']
L = loaded_variables['L']
paths = loaded_variables['paths']
sector_labels = loaded_variables['sector_labels']
total_impact = loaded_variables['total_impact']
centrality = loaded_variables['centrality']
bonacich_centrality = loaded_variables['bonacich_centrality']

# Ensure Eora_pp_FD is a NumPy array
if not isinstance(Eora_pp_FD, np.ndarray):
    Eora_pp_FD = np.array(Eora_pp_FD)

# Initial conditions for production and emissions
initial_production = Eora_pp_T_matrix.sum(axis=1)  # Initial production values (1D array)
initial_emissions = Eora_pp_Q_emissions_matrix.sum(axis=1)  # Initial emissions values (1D array)

# Combine into a single initial conditions vector
initial_conditions = np.concatenate([initial_production, initial_emissions])

def feedback_production(production, demand_matrix):
    return np.dot(demand_matrix, production)

def feedback_emissions(production, emissions_factors):
    return emissions_factors * production

def system_dynamics(y, t, demand_matrix, emissions_factors, external_shocks):
    num_sectors = demand_matrix.shape[0]
    production = y[:num_sectors]
    emissions = y[num_sectors:]

    production_change = feedback_production(production, demand_matrix) + external_shocks(t, num_sectors)
    emissions_change = feedback_emissions(production, emissions_factors)

    return np.concatenate([production_change, emissions_change])

def external_shocks(t, num_sectors):
    # Example external shock function
    return np.sin(t) * np.ones(num_sectors)

# Time vector for simulation
t = np.linspace(0, 10, 100)

# Demand matrix and emissions factors
demand_matrix = Eora_pp_FD
emissions_factors = f

# Perform the simulation
result = odeint(system_dynamics, initial_conditions, t, args=(demand_matrix, emissions_factors, external_shocks))

# Extract results
num_sectors = demand_matrix.shape[0]
production_result = result[:, :num_sectors]
emissions_result = result[:, num_sectors:]

# Plot results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
for i in range(num_sectors):
    plt.plot(t, production_result[:, i], label=f'Sector {i+1}')
plt.title('Production over time')
plt.xlabel('Time')
plt.ylabel('Production')
plt.legend()

plt.subplot(1, 2, 2)
for i in range(num_sectors):
    plt.plot(t, emissions_result[:, i], label=f'Sector {i+1}')
plt.title('Emissions over time')
plt.xlabel('Time')
plt.ylabel('Emissions')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Filter Emissions Rows
emission_indices = labels_Q[labels_Q.iloc[:, 0].str.contains(r"\(Gg\)") & ~labels_Q.iloc[:, 0].str.contains("Total")].index

# Subset the Emissions Data
Eora_pp_Q_emissions = Eora_pp_Q.iloc[emission_indices]

# Transform data frames into matrices
Eora_pp_T_matrix = Eora_pp_T.values
Eora_pp_Q_emissions_matrix = Eora_pp_Q_emissions.values

# Get the emission labels
emission_labels = labels_Q.iloc[emission_indices, 0].values

# Get the sector labels (assuming labels_T contains sector labels for columns)
sector_labels = labels_T.iloc[:, 3].values  # Adjust this index if the sector labels are in a different column

# Create a DataFrame with the emissions matrix, using emission labels as row index and sector labels as column headers
Eora_pp_Q_emissions_df = pd.DataFrame(Eora_pp_Q_emissions_matrix, index=emission_labels, columns=sector_labels)

# Investigate the DataFrame
print(f"Shape of Eora_pp_Q_emissions_df: {Eora_pp_Q_emissions_df.shape}")
print("First 5 rows and columns of the emissions matrix with labels:")
print(Eora_pp_Q_emissions_df.iloc[:5, :5])

# Summary statistics
print("Summary statistics:")
print(Eora_pp_Q_emissions_df.describe())

# Emission labels (first 10)
print("Emission labels (first 10):")
print(Eora_pp_Q_emissions_df.index[:10])

# Sector labels (first 10)
print("Sector labels (first 10):")
print(Eora_pp_Q_emissions_df.columns[:10])


Shape of Eora_pp_Q_emissions_df: (2047, 4915)
First 5 rows and columns of the emissions matrix with labels:
                          Agriculture  Fishing  Mining and Quarrying  \
I-GHG-CO2 emissions (Gg)        1.575    1.575                1.5750   
I-GHG-CO2 emissions (Gg)        1.575    1.575                1.5750   
I-GHG-CO2 emissions (Gg)        1.575    1.575                1.8744   
I-GHG-CO2 emissions (Gg)        1.575    1.575                1.5750   
I-GHG-CO2 emissions (Gg)        1.575    1.575                1.5750   

                          Food & Beverages  Textiles and Wearing Apparel  
I-GHG-CO2 emissions (Gg)           1.57500                       1.57500  
I-GHG-CO2 emissions (Gg)           1.57500                       1.57500  
I-GHG-CO2 emissions (Gg)           3.11217                       1.85569  
I-GHG-CO2 emissions (Gg)           1.57500                       1.57500  
I-GHG-CO2 emissions (Gg)           1.57500                       1.57500  
Summary s

In [ ]:
country_labels = labels_T.iloc[:, 1].values
print(country_labels)

NameError: name 'results' is not defined